In [ ]:
import re, os, json
from openai import OpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

In [ ]:
SYSTEM_PROMPT  = """
너의 역할은 '도메인 용어 최종 필터'이다.

입력은 question, answer 로 구성된 하나의 용어 항목이다.
이 항목이 IT/소프트웨어/통신/네트워크/데이터/보안/AI/서비스/플랫폼 등
실제 기술 회의에서 자주 등장할 가능성이 높은 핵심 도메인 용어이면 keep=true,
그렇지 않으면 keep=false 로 분류하라.

중요:
- "너무 일반적이거나, 기초적인 상식 수준의 개념"이라면 keep=false 로 분류한다.
- 즉, 전문적 기술 분야와 직접적으로 연결되지 않거나,
  소규모 모델(예: 1~5B급 일반 언어 모델)도 무난하게 이해할 수 있는 기본 개념이라면 keep=false 로 본다.

출력은 JSON 객체 하나로만 출력하며,
keep 필드에 JSON boolean(true/false) 값만 포함해야 한다.
자연어 설명 금지.
"""

def gpt_is_domain_term(question: str, answer: str) -> bool:
    user_prompt = f"""
question: {question}
answer: {answer}
"""

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",    "content": user_prompt},
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "domain_classification",
                "schema": {
                    "type": "object",
                    "properties": {
                        "keep": { "type": "boolean" }
                    },
                    "required": ["keep"],
                    "additionalProperties": False
                }
            },
        },
        temperature=0,
        max_tokens=10,
    )
    return json.loads(resp.choices[0].message.content)['keep']

In [ ]:
target_path = "TTA_scaling_v5"
os.makedirs(target_path, exist_ok=True)

In [ ]:
folder_path = 'TTA_scaling_v4'
folder_list = os.listdir(folder_path)
folder_list

In [ ]:
for file in folder_list:
    if '_sorted_종합_' not in file:
        continue
    file_path = os.path.join(folder_path, file)
    print(file_path)
    with open(file_path, "r", encoding="utf-8") as f:
        with open(os.path.join(target_path, "gpt_keep_" + file), 'a', encoding="utf-8") as f_keep:
            with open(os.path.join(target_path, "gpt_drop_"+ file), 'a', encoding="utf-8") as f_drop:
                for line in f:
                    data = json.loads(line)['messages']
                    print('Q:', data[0]['content']+'\nA:',data[1]['content'])
                    print(json.dumps(data, ensure_ascii=False))
                    t_f = gpt_is_domain_term(data[0]['content'], data[1]['content'])
                    if (t_f):
                        f_keep.write(json.dumps(data, ensure_ascii=False) + '\n')
                    else:
                        f_drop.write(json.dumps(data, ensure_ascii=False) + '\n')